# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [32]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import inauguralproject
import HouseholdSpecializationModel

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

In [33]:
# code for solving the model (remember documentation and comments)

a = np.array([1,2,3])
#b = inauguralproject.square(a)
#print(b)

model = HouseholdSpecializationModel.HouseholdSpecializationModelClass()



HM = np.zeros([3,3])

alphalist = [0.25, 0.50, 0.75]
sigmalist = [0.5, 1.0, 1.5]

for i, j in enumerate(alphalist):
    model.par.alpha = j
    for it, s in enumerate(sigmalist):
        model.par.sigma = s
        out =  model.solve_discrete()
        HM[i,it] = out.HF /out.HM



[1 4 9]


/Users/emmagabriel/Documents/KU/6. Semester/Introduction to Programming and Numerical Analysis/projects-2023-hold-11/inauguralproject/HouseholdSpecializationModel.py:64: RuntimeWarning: divide by zero encountered in reciprocal
  H = ((1-par.alpha)*HM**((par.sigma-1)/par.sigma)+par.alpha*HF**((par.sigma-1)/par.sigma))**(par.sigma/(par.sigma-1))
/Users/emmagabriel/Documents/KU/6. Semester/Introduction to Programming and Numerical Analysis/projects-2023-hold-11/inauguralproject/HouseholdSpecializationModel.py:64: RuntimeWarning: invalid value encountered in reciprocal
  H = ((1-par.alpha)*HM**((par.sigma-1)/par.sigma)+par.alpha*HF**((par.sigma-1)/par.sigma))**(par.sigma/(par.sigma-1))
/Users/emmagabriel/Documents/KU/6. Semester/Introduction to Programming and Numerical Analysis/projects-2023-hold-11/inauguralproject/HouseholdSpecializationModel.py:73: RuntimeWarning: invalid value encountered in reciprocal
  utility = np.fmax(Q,1e-8)**(1-par.rho)/(1-par.rho)


In [34]:
#Resets parameters
model.alpha = 0.5
model.sigma=1

# Question 2

In [35]:
model.solve_wF_vec(discrete=True)

/Users/emmagabriel/Documents/KU/6. Semester/Introduction to Programming and Numerical Analysis/projects-2023-hold-11/inauguralproject/HouseholdSpecializationModel.py:73: RuntimeWarning: invalid value encountered in reciprocal
  utility = np.fmax(Q,1e-8)**(1-par.rho)/(1-par.rho)


In [43]:
model.sol.LM_vec
model.sol.HM_vec
model.sol.LF_vec
model.sol.HF_vec

array([8. , 7.5, 7. , 7. , 6.5])

In [ ]:
import matplotlib.pyplot as plt



Explain your code and procedure

In [ ]:
blabla = np.zeros([1,5])
wFlist = [0.8, 0.9, 1.0 , 1.1, 1.2]

for i, w in enumerate(wFlist):
    model.par.wF_vec = w
    out = model.solve_discrete()
    blabla[i] = np.log(out.HF / out.HM)

In [ ]:
HM_log = np.log(HM)
par.wM = 1.0
par.wF = 1.0
par.wF_vec = np.linspace(0.8,1.2,5)

bla = np.log(par.wF_vec/par.wM)

import matplotlib.pyplot as plt
plt.plot(HM_log, bla)
plt.show

# Question 3

Explain your code and procedure

In [ ]:
# code

ADD CONCISE CONLUSION.

In [ ]:
#Question 4
